In [1]:
!nvidia-smi

Fri Mar 31 01:40:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.141.03   Driver Version: 470.141.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:8A:00.0 Off |                    0 |
| N/A   29C    P0    44W / 300W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# # mount driver
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/CardiacSeg/eda/Network
#
# #install dependents
# !pip install -q "monai-weekly[nibabel, tqdm, einops]"
# !python -c "import matplotlib" || pip install -q matplotlib
# !pip install -q timm
# !pip install -U -q openmim
# !mim install -U -q mmcv-full
# !pip install ml-collections
# !pip install torchsummaryX
# !pip install ray
# %matplotlib inline

In [2]:
import sys

from torchsummaryX import summary

import torch

# set package path
sys.path.append(r'/nfs/Workspace/CardiacSeg')

import argparse
from networks.network import network
from networks.networkx.blocks.cbam import CBAM
from networks.ssl_head import SSLHead

# sync python module
%load_ext autoreload
%autoreload 2

/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
import torch
from torch import nn

model = nn.Linear(10, 1)

# import Lion and instantiate with parameters

from lion_pytorch import Lion

opt = Lion(model.parameters(), lr=1e-4, weight_decay=1e-5)

In [5]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [128, 128, 128]
args.patch_size = 4
args.drop_rate = 0.4
args.depths = [4, 4, 8, 4]
net = None
del net
net = network('unest', args)
inp = None
del inp
inp = torch.rand((1, 1, 128, 128, 128)).to(args.device)
summary(net, inp)

model: unest
                                                            Kernel Shape  \
Layer                                                                      
0_nestViT.patch_embed.Conv3d_proj                      [1, 128, 4, 4, 4]   
1_nestViT.levels.0.Identity_pool                                       -   
2_nestViT.levels.0.transformer_encoder.0.LayerN...                 [128]   
3_nestViT.levels.0.transformer_encoder.0.attn.L...            [128, 384]   
4_nestViT.levels.0.transformer_encoder.0.attn.D...                     -   
5_nestViT.levels.0.transformer_encoder.0.attn.L...            [128, 128]   
6_nestViT.levels.0.transformer_encoder.0.attn.D...                     -   
7_nestViT.levels.0.transformer_encoder.0.Identi...                     -   
8_nestViT.levels.0.transformer_encoder.0.LayerN...                 [128]   
9_nestViT.levels.0.transformer_encoder.0.mlp.Li...            [128, 512]   
10_nestViT.levels.0.transformer_encoder.0.mlp.G...                     -   

The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_nestViT.patch_embed.Conv3d_proj,"[1, 128, 4, 4, 4]","[1, 128, 32, 32, 32]",8320.0,2.684355e+08
1_nestViT.levels.0.Identity_pool,-,"[1, 128, 32, 32, 32]",NaN,NaN
2_nestViT.levels.0.transformer_encoder.0.LayerNorm_norm1,[128],"[1, 64, 512, 128]",256.0,1.280000e+02
3_nestViT.levels.0.transformer_encoder.0.attn.Linear_qkv,"[128, 384]","[1, 64, 512, 384]",49536.0,4.915200e+04
4_nestViT.levels.0.transformer_encoder.0.attn.Dropout_attn_drop,-,"[1, 4, 64, 512, 512]",NaN,NaN
...,...,...,...,...
232_decoder1.conv_block.InstanceNorm3d_norm2,-,"[1, 32, 128, 128, 128]",NaN,NaN
233_decoder1.conv_block.conv3.Conv3d_conv,"[64, 32, 1, 1, 1]","[1, 32, 128, 128, 128]",2048.0,4.294967e+09
234_decoder1.conv_block.InstanceNorm3d_norm3,-,"[1, 32, 128, 128, 128]",NaN,NaN


In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [128, 128, 128]
net = None
del net
net = network('unetcnx_x2', args)
inp = None
del inp
inp = torch.rand((1, 1, 128, 128, 128)).to(args.device)
summary(net, inp)

model: unetcnx_x2


/usr/local/lib/python3.8/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


                                                           Kernel Shape  \
Layer                                                                     
0_encoder0.layer.conv1.Conv3d_conv                     [1, 24, 3, 3, 3]   
1_encoder0.layer.InstanceNorm3d_norm1                                 -   
2_encoder0.layer.LeakyReLU_lrelu                                      -   
3_encoder0.layer.conv2.Conv3d_conv                    [24, 24, 3, 3, 3]   
4_encoder0.layer.InstanceNorm3d_norm2                                 -   
5_encoder0.layer.conv3.Conv3d_conv                     [1, 24, 1, 1, 1]   
6_encoder0.layer.InstanceNorm3d_norm3                                 -   
7_encoder0.layer.LeakyReLU_lrelu                                      -   
8_inc.Conv3d_0                                         [1, 48, 2, 2, 2]   
9_inc.Conv3d_0                                         [1, 48, 2, 2, 2]   
10_inc.LayerNorm3d_1                                               [48]   
11_inc.LayerNorm3d_1     

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_encoder0.layer.conv1.Conv3d_conv,"[1, 24, 3, 3, 3]","[1, 24, 128, 128, 128]",648.0,1.358954e+09
1_encoder0.layer.InstanceNorm3d_norm1,-,"[1, 24, 128, 128, 128]",NaN,NaN
2_encoder0.layer.LeakyReLU_lrelu,-,"[1, 24, 128, 128, 128]",NaN,NaN
3_encoder0.layer.conv2.Conv3d_conv,"[24, 24, 3, 3, 3]","[1, 24, 128, 128, 128]",15552.0,3.261491e+10
4_encoder0.layer.InstanceNorm3d_norm2,-,"[1, 24, 128, 128, 128]",NaN,NaN
...,...,...,...,...
387_decoder1.decoder_block.0.LeakyReLU_lrelu,-,"[1, 24, 128, 128, 128]",NaN,NaN
388_decoder1.decoder_block.0.conv2.Conv3d_conv,"[24, 24, 3, 3, 3]","[1, 24, 128, 128, 128]",15552.0,3.261491e+10
389_decoder1.decoder_block.0.InstanceNorm3d_norm2,-,"[1, 24, 128, 128, 128]",NaN,NaN


## DenseVoxNet

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
net = network('dense_vox_net', args)
inp = torch.rand((1, 1, 96, 96, 96)).to(args.device)
summary(net, inp)

## EffcientSegNet

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
net = network('efficient_segnet_x0_2', args)
summary(net, (1, 96, 96, 96))

## UNETCNX

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
net = network('unetcnx_x0', args)
model = SSLHead(args.in_channels, net).to(args.device)
summary(model, (1, 96, 96, 96))

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
model = network('unetcnx_x1', args)
summary(model, (1, 96, 96, 96))